# Bernstein-Vazirani algorithm(ベルンシュタイン・ヴァジラニ アルゴリズム)（概要）

Bernstein-Vazirani algorithm を説明します。

このアルゴリズムでは $x$ を入力とし、内部に持つビット列 $a$ を用いて出力を計算する次のような関数を考えます。

$$
f_a(x) = (a\cdot x) \bmod2 = (\sum_i a_i x_i) \bmod2
$$

目的は、$f_a(x)$ の出力からビット列 $a$ を求めることです。

ただし、$a, x$ はそれぞれ $n$ビット列 $a = a_0 a_1... a_{n-1}$, $x = x_0 x_1... x_{n-1}$ と表します。

$a$ が4ビット列 $1001$ の場合を考えましょう。  
古典計算における愚直な求め方は、$x=1000, 0100, 0010, 0001$ のように1つのビットだけを1とし、他を0とした $x$ を順に入力します。   
$0 \cdot 0 = 0 \cdot 1  = 0$ より $x_i=0$としたビットは結果に影響しないため、$x_i = 1$ とした1ビットずつ $a_i$ の値を決定することができます。  

$a=1001$ なる $f_a(x)$ を、4通りの $x$ について計算すると以下のようになります。

$(1001 \cdot 1000) \bmod2 = 1$  
$(1001 \cdot 0100) \bmod2 = 0$  
$(1001 \cdot 0010) \bmod2 = 0$  
$(1001 \cdot 0001) \bmod2 = 1$

以上の出力より $a=1001$ を得ることができます。

Bernstein-Vazirani algorithm は量子の重ね合わせを用いて1度の計算で $a$ を求めます。  

$a=1001$ の場合の具体的な回路は次の通りです。  

<img src="./img/102_img.png" width="30%">

$a_i=1$ である量子ビット $i$ をそれぞれ制御量子ビットとした、$CX$ゲートを作用させます。  
ターゲット量子ビットは全て4番目の補助量子ビットです。

それぞれの状態を確認します。

$$
\begin{align}
\lvert \psi_1\rangle &= \biggl(\otimes^4 H\lvert 0\rangle \biggr)\otimes H \lvert 1\rangle \\
&= \frac{1}{\sqrt{2}}(\lvert 0\rangle + \lvert 1\rangle) \otimes \frac{1}{\sqrt{2}}(\lvert 0\rangle + \lvert 1\rangle) \otimes \frac{1}{\sqrt{2}}(\lvert 0\rangle + \lvert 1\rangle) \otimes \frac{1}{\sqrt{2}}(\lvert 0\rangle + \lvert 1\rangle) \otimes \frac{1}{\sqrt{2}}(\lvert 0\rangle - \lvert 1\rangle)
\end{align}
$$

次に $\lvert \psi_2\rangle$ の状態を確認します。  
$CX$ゲートを作用させる0番目と4番目の量子ビットだけに注目してみましょう。

$$
\begin{align}
\lvert \psi_1\rangle_{04} &= \frac{1}{\sqrt{2}}(\lvert 0\rangle_0 + \lvert 1\rangle_0) \otimes \frac{1}{\sqrt{2}}(\lvert 0\rangle_4 - \lvert 1\rangle_4) \\
&= \lvert +\rangle_0 \otimes \lvert -\rangle_4
\end{align}
$$

$$
\begin{align}
\lvert \psi_2\rangle_{04} &= \frac{1}{\sqrt{2}}\lvert 0\rangle_0 \otimes \frac{1}{\sqrt{2}}(\lvert 0\rangle_4 - \lvert 1\rangle_4) + \frac{1}{\sqrt{2}}\lvert 1\rangle_0 \otimes \frac{1}{\sqrt{2}}(\lvert 1\rangle_4 - \lvert 0\rangle_4) \\
&= \frac{1}{\sqrt{2}}\lvert 0\rangle_0 \otimes \frac{1}{\sqrt{2}}(\lvert 0\rangle_4 - \lvert 1\rangle_4) - \frac{1}{\sqrt{2}}\lvert 1\rangle_0 \otimes \frac{1}{\sqrt{2}}(\lvert 0\rangle_4 - \lvert 1\rangle_4) \\
&= \frac{1}{\sqrt{2}}(\lvert 0\rangle_0 - \lvert 1\rangle_0 ) \otimes \frac{1}{\sqrt{2}}(\lvert 0\rangle_4 - \lvert 1\rangle_4) \\
&=  \lvert -\rangle_0 \otimes \lvert -\rangle_4
\end{align}
$$



0番目と4番目の量子ビットに$CX$ゲートを作用させることで、0番目の量子ビットの位相を反転させて$\lvert + \rangle \to \lvert - \rangle$と変化させました。  
同様に3番目と4番目の量子ビットに$CX$ゲートを作用させた状態 $\lvert \psi_3\rangle$ は次の通りです。

$$
\lvert \psi_3\rangle = \frac{1}{\sqrt{2}}(\lvert 0\rangle - \lvert 1\rangle) \otimes \frac{1}{\sqrt{2}}(\lvert 0\rangle + \lvert 1\rangle) \otimes \frac{1}{\sqrt{2}}(\lvert 0\rangle + \lvert 1\rangle) \otimes \frac{1}{\sqrt{2}}(\lvert 0\rangle - \lvert 1\rangle) \otimes \frac{1}{\sqrt{2}}(\lvert 0\rangle - \lvert 1\rangle)
$$

最後にHゲートを作用させます。

$$
\lvert \psi_4\rangle = \lvert 1\rangle \otimes \lvert 0\rangle \otimes \lvert 0\rangle \otimes \lvert 1\rangle \otimes \frac{1}{\sqrt{2}}(\lvert 0\rangle - \lvert 1\rangle)
$$

0番目から3番目の量子ビットを測定すると、$1001$が測定結果として得られます。  
以上で$a$を求めることができました。

これをblueqatで実装してみましょう。

In [1]:
from blueqat import Circuit
import numpy as np

まず、オラクル$U_f$を作用させる関数を用意します。

In [ ]:
def Uf(c, a):
    N = len(a)
    for i, val in enumerate(list(a)):
        if val == '1':
            c.cx[i, len(a)]

以下がアルゴリズム本体です。  
最初に、求めたい $a$ を乱数で決めます。

オラクルを用いた量子回路の出力結果から $a$ を求め、正解かを確認します。

In [67]:
n = 4

a = ''
for i in range(n):
    a += str(np.random.randint(2))
    
c = Circuit(n + 1)
c.x[n].h[:]
Uf(c, a)
c.h[:].m[:]
res = c.run(shots = 1000)

print(res)

if [arr[:n] for arr in res.keys()] == [a]:
    print("OK")
else:
    print("incorrect")

Counter({'11001': 1000})
OK


以上より、Bernstein-Vazirani algorithm によってオラクルが内部的に持つ $a$ を求めることができました。